In [ ]:
import torch

import matplotlib.pyplot as plt
import torchmetrics as tm

from json import load, dump

from config import *
from stages import *
from train import *
from data.ssa import SSA

from data.util import crop_q_between, split_weekdays_and_weekends

%reload_ext autoreload
%autoreload 2

In [ ]:
train_portion = CONFIG.data_split[0]
mse = tm.MeanSquaredError().to(CONFIG.device)
mape = tm.MeanAbsolutePercentageError().to(CONFIG.device)
mat_q = CONFIG.load('mat_q.pt')
CONFIG.alpha = 0.2
CONFIG.alpha = 1000
mat_c_all, mat_x_repr, nonempty, representatives = compress_data(
    mat_q.abs(), CONFIG.read_period, CONFIG.test_period, CONFIG.alpha)
mat_q = mat_q[:, mat_q.sum(dim=0) > 0]
normalizing_constants_all = mat_q.max(dim=0)[0]
# mat_q /= normalizing_constants_all
mat_q

In [ ]:
results_wd = {}
results_we = {}

ssa = SSA(4, [2, 2], CONFIG.device)

CONFIG.spectral_threshold = 1440
CONFIG.dbn_hidden_layer_sizes = [80, 80, 80]

gamma = 1
reg_coeff = 1

CONFIG.data_split = [0.0, 1.0]

for gamma in [gamma]:#range(1, 26, 5):
  for reg_coeff in [reg_coeff]:#range(1, 102, 25):
    for P in [70]:#tqdm(range(110, 210, 10)):#[40, 126.5, 400, 1265, 4000, 12650, 40000]):#range(125, 3125, 125):
      results_P = {}
      CONFIG.spectral_threshold = P
      for N in [84,]:#range(4, 324, 80):
        CONFIG.dbn_hidden_layer_sizes = [N, N, N]

        mat_c_all, mat_x_all, nonempty, representatives = compress_data(
            mat_q.abs(), CONFIG.read_period, CONFIG.test_period, CONFIG.alpha)
        mat_q_trend_all, mat_q_resid_all = preprocess_data(
            CONFIG.spectral_threshold, mat_c_all)
        mat_q_resid_all = crop_q_between(mat_q_resid_all, CONFIG.read_period,
                                         CONFIG.test_period).real
        mat_c_all = crop_q_between(
            mat_c_all, CONFIG.read_period, CONFIG.test_period)

        representatives = torch.tensor([160,])
        for section_index, section_number in tqdm(enumerate(representatives.tolist())):
          mat_q_trend = mat_q_trend_all[:, section_index][:, None].real
          mat_q_resid = mat_q_resid_all[:, section_index][:, None].real

          norm_const = normalizing_constants_all[section_number].item()

          (_, val_trend_wd_dataloader, _), (_, val_trend_we_dataloader, _) = \
            crop_and_split_mat(mat_q_trend, CONFIG, test_period=True, separate_weekends=False)
          # (_, val_resid_wd_dataloader, _), (_, val_resid_we_dataloader, _) =
          #   crop_and_split_mat(mat_q_resid, CONFIG)
          mat_c_wd_datasets, mat_c_we_datasets = split_mat(mat_q_resid, CONFIG)
          # print(gamma, reg_coeff, P, N, mat_c.shape)

          del mat_q_resid

          # dbn, kelm, val_resid_we_dataloader = train_with_config(
          #     CONFIG, mat_c_we_datasets, dbn_training_epochs=100, stride=1,
          #     gamma=gamma, reg_coeff=reg_coeff)
          # val_resid_we_dataloader = DataLoader(mat_c_we_datasets[1])
          val_resid_wd_dataloader = DataLoader(mat_c_wd_datasets[1])
          prefix = f'alpha{CONFIG.alpha}_p{P}_n{N}_sec{section_number}_'\
            .replace('.', '_')
          dbn = DBN(CONFIG.time_window_length, CONFIG.dbn_hidden_layer_sizes,
            CONFIG.gibbs_sampling_steps).to(CONFIG.device)
          dbn.load_state_dict(CONFIG.load(prefix + 'dbn.pt'))
          
          train_len = int(len(mat_c_wd_datasets[1]) * train_portion) - 2
          kelm_state = CONFIG.load(prefix + 'kelm.pt')
          kelm = KELM(kelm_state['_beta'].shape, kelm_state['_data'].shape).to(CONFIG.device)
          kelm.gamma = 1.0
          kelm.reg_coeff = 1.0
          kelm.load_state_dict(kelm_state)

          mse_loss_trend = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_resid = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_overall = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_trend_unnorm = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_resid_unnorm = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_overall_unnorm = torch.tensor([0.,]).to(CONFIG.device)

          trend_y = torch.tensor([0.,]).to(CONFIG.device)
          resid_y = torch.tensor([0.,]).to(CONFIG.device)
          overall_y = torch.tensor([0.,]).to(CONFIG.device)

          iter_trend = iter(val_trend_wd_dataloader)
          n_samples = 0

          resid = {'y': [], 'pred': []}
          trend = {'y': [], 'pred': []}
          overall = {'y': [], 'pred': []}
          for X_resid, y_resid in val_resid_wd_dataloader:
              resid_y += y_resid.item()
              X_trend, y_trend = next(iter_trend)
              trend_y += y_trend.item()
              pred_trend = ssa.forecast(
                  X_trend.squeeze(0).T, 1).sum(0)[-1][None]
              mse_loss_trend += mse(pred_trend, y_trend)
              mse_loss_trend_unnorm += mse(pred_trend * norm_const, y_trend * norm_const)
              trend['y'].append(y_trend.item() * norm_const)
              trend['pred'].append(pred_trend.item() * norm_const)
              # print('TREND', y_trend, pred_trend)

              pred_resid = dbn(X_resid).squeeze(0)
              pred_resid = kelm(pred_resid).T
              mse_loss_resid += mse(pred_resid, y_resid)
              mse_loss_resid_unnorm += mse(pred_resid * norm_const, y_resid * norm_const)
              resid['y'].append(y_resid.item() * norm_const)
              resid['pred'].append(pred_resid.item() * norm_const)

              # print('RESID', y_resid, pred_resid)

              pred = pred_trend + pred_resid

              y = y_trend + y_resid
              overall_y += y.item()
              overall['y'].append(y.item() * norm_const)
              overall['pred'].append(pred.item() * norm_const)
              # print('OVRLL', y, pred)
              # print('=====')

              mse_loss_overall += mse(pred, y).item()
              mse_loss_overall_unnorm += mse(pred * norm_const, y * norm_const).item()
              n_samples += 1

          mse_loss_trend /= n_samples
          mse_loss_resid /= n_samples
          mse_loss_overall /= n_samples
          mse_loss_resid_unnorm /= n_samples
          mse_loss_trend_unnorm /= n_samples
          mse_loss_overall_unnorm /= n_samples

          resid_y /= n_samples
          trend_y /= n_samples
          overall_y /= n_samples

          results_P[section_number] = {
              'resid': resid,
              'trend': trend,
              'overall': overall
          }

          with open(f'results_wd_{section_number}.json', 'w') as file:
            dump(results_P[section_number], file)

          # print(f'gamma={gamma}, reg_coeff={reg_coeff}, ' +
          #       f'WE P={CONFIG.spectral_threshold}, ' +
          #       f'N={CONFIG.dbn_hidden_layer_sizes}, ' +
          #       f'trend={trend_y.item()}+-{mse_loss_trend.item()}' +
          #       f'~{mse_loss_trend_unnorm.item()}, ' +
          #       f'resid={resid_y.item()}+-{mse_loss_resid.item()}' +
          #       f'~{mse_loss_resid_unnorm.item()}, ' +
          #       f'overall={overall_y.item()}+-{mse_loss_overall.item()}' +
          #       f'~{mse_loss_overall_unnorm.item()}')
          del dbn
          del kelm
      results_wd[P] = results_P

In [ ]:
from json import dump
with open('results_we_16.json', 'w') as file:
    dump(results_P, file)